In [22]:
import pandas as pd
import ssl 
import numpy as np
import os
import requests
import logging
from bs4 import BeautifulSoup
from pathlib import Path
import time

ssl._create_default_https_context = ssl._create_unverified_context


Stores and retreives player photo links

In [23]:
def player_pic(p_id):
     pic_dict={
          '253802' : 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320,q_50/lsci/db/PICTURES/CMS/316600/316605.png',
          '34102' : 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320,q_50/lsci/db/PICTURES/CMS/316500/316584.png',
          '422108' : 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320,q_50/lsci/db/PICTURES/CMS/319900/319942.png',
          '28235' : 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320,q_50/lsci/db/PICTURES/CMS/316500/316524.png',
          '28081' : 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_640,q_50/lsci/db/PICTURES/CMS/319900/319946.png',
          '234675' : 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320,q_50/lsci/db/PICTURES/CMS/316600/316600.png',
          '625371' : 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320,q_50/lsci/db/PICTURES/CMS/319900/319938.png',
          '26421' : 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320,q_50/lsci/db/PICTURES/CMS/316500/316521.png',          
          '625383' : 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320,q_50/lsci/db/PICTURES/CMS/319900/319940.png',
          '481896' : 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320,q_50/lsci/db/PICTURES/CMS/316600/316691.png',
          '326016' : 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320,q_50/lsci/db/PICTURES/CMS/316600/316658.png',
          '376116' : 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320,q_50/lsci/db/PICTURES/CMS/316600/316667.png'
     }
     return pic_dict.get(p_id)

Table: 'player_overview'

In [24]:
def player_overview(player_ids):
    
    player_overview_merged = pd.DataFrame()

    for p_id in player_ids:
        
        # URL of the website to scrape
        url = "https://www.espncricinfo.com/cricketers/x-"+p_id

        # Send an HTTP GET request to the website
        response = requests.get(url)

        # Parse the HTML code using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the relevant information from the HTML code
        class_grid='ds-grid lg:ds-grid-cols-3 ds-grid-cols-2 ds-gap-4 ds-mb-8'
        div_p = soup.find('div', {'class': class_grid})
        
        my_dict = {}
        headers=['Full Name', 'Born', 'Age', 'Batting Style', 'Bowling Style', 'Playing Role']

        class_para = 'ds-text-tight-m ds-font-regular ds-uppercase ds-text-typo-mid3'
        for div in div_p.find_all('div'):
            if ( div.find('p', {'class':class_para}) ) is None:
                continue

            key = div.find('p', {'class':class_para}).text
            value = div.find('span').text
            if key in headers:
                my_dict[key] = value

        #Adds players' roll ids: (ALL for All rounder), (BAT for Batsman), (BOW for Bowler)
        if 'All' in my_dict.get('Playing Role'):
            my_dict['Roll_id'] = 'ALL'
        elif 'Bowl' in my_dict.get('Playing Role'):
            my_dict['Roll_id'] = 'BOW'
        elif 'Bat' in my_dict.get('Playing Role'):
            my_dict['Roll_id'] = 'BAT'
        else:
            log.error('No matching playing role is found')

        #Adds link of the player picture
        my_dict['photo_link'] = player_pic(p_id)

        player_overview = pd.DataFrame.from_dict([my_dict])
        player_overview.insert(0, "P_id", p_id, True)

        player_overview_merged = pd.concat([player_overview_merged, player_overview], ignore_index=True)
        time.sleep(0.1)

    filepath = Path('Csv/player_overview.csv')
    player_overview_merged.to_csv(filepath, index_label="row_id")

    print(player_overview_merged)

#Main function below:

In [25]:
#253802: Virat Kohli (BAT)
#34102: Rohit Gurunath Sharma (BAT)
#422108: Kannaur Lokesh Rahul (BAT)
#28235: Shikhar Dhawan (BAT)
#28081 : Mahendra Singh Dhoni (BAT)
#234675: Ravindrasinh Anirudhsinh Jadeja (ALL)
#625371: Hardik Himanshu Pandya (ALL)
#26421: Ravichandran Ashwin (ALL)
#625383: Jasprit Jasbirsingh Bumrah (BOW)
#481896: Mohammed Shami Ahmed (BOW)
#326016: Bhuvneshwar Kumar Singh (BOW)
#376116: Umeshkumar Tilak Yadav (BOW)


player_ids=['253802', '34102', '422108', '28235', '28081', '625371', '26421', '625383', '481896', '326016', '376116']

os.system("rm -f Csv/player_overview.csv")

player_overview(player_ids)

